<p style="text-align:center"><font size="10" >Kawase Seminar 06-2</font></p>

全2回にわけて実際の論文の手法を実践し、スクレイピングとテキストマイングの利用を体験していきます。

第6回でも、1.実際に現状を把握する, 2.解くための筋道を立てる, 3.Densuke(田村)が実装したコードを読んで2の筋道が実際に正しかったのか確認する, といったことを逐次行っていきます。

今回以降は、時間の関係上実装はそこまで行えませんがかなりの学びになるので、皆さんも各自でトライして頂けると良いと思われます💪

### **やること**
-  テキストマイニング分野における実装を大まかに考える
-  大まかな実装を細かい単位に分解し具体的なロジックを組み立てる
-  **ロジックをもとに実装する ->イマココ**


今回のコードを読む目的としては、どのように田村が実装したのかを田村の思考を理解し、

実際に自分で書く際の糧になってもらいことにあります。


---
## **田村のTODOメモ**

1. 第１段階、第２段階の各段階で収集したメッセージデータの中から1000個のメッセージをランダムに選択
    1.  ランダムに企業コードを選択するプログラムを実行し、選択された企業名からcsvデータを読み込む
    2. 読み込まれたcsvデータのメッセージの項目から適当に1つ選択する
    3. 1, 2を1000回繰り返す
　
2. 1000個のメッセージを手動で強気、弱気、中立に分類

3. ２で、分類されたメッセージの中から、「売り」「買い」「高」など、強気・弱気の感情に大きく関係するキーワードを選択。

4. macabなどの形態素解析ライブラリでメッセージの単語を分割する(その際に情報やノイズの多い情報を除外するために167の単語を辞書に追加)

5. 企業数分のメッセージ（形態素解析で分けられた単語群）配列をキーワード頻度行列に変換

6. SVM利用し、3つのセンチメントクラスに分類

___


## 実装が可能な部分を踏まえたTODO

今回、時間内や調査の関係上実装が難しいところがあるため、内容を変更してTODOを再作成しました。

1. 企業のcsvデータから、macabなどの形態素解析ライブラリでメッセージの単語を分割
2. メッセージをplus(1>=N),  minus(N<= -1 ), normal(±0) に分類
3.  plus, minus, normalに分けたメッセージにある単語を多い順にソートする

✴︎今回はSVMで学習させて分類するのではなく、すでに学習されたモデル(日本語評価極性辞書)を利用し分類
https://www.cl.ecei.tohoku.ac.jp/Open_Resources-Japanese_Sentiment_Polarity_Dictionary.html

---
# 1. 企業のcsvデータから、macabなどの形態素解析ライブラリでメッセージの単語を分割

> 形態素解析とは、自然言語処理（NLP）の一部で、自然言語で書かれた文を言語上で意味を持つ最小単位(＝形態素)に分け、それぞれの品詞や変化などを判別することです。
例えば　「庭には二羽ニワトリがいる」という文章を庭（名詞）/に（助詞）/は（助詞）/二（数詞）/羽（助数詞）/ニワトリ（名詞）/が（助詞）/いる（動詞）のように形態素に分解し、意味を割り出します。
https://ledge.ai/morpho_analysis_japan/


今回は形態素解析の方法の一つとしてオープンソースのmecabを利用
https://qiita.com/taroc/items/b9afd914432da08dafc8

In [1]:
# https://qiita.com/pytry3g/items/897ae738b8fbd3ae7893
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [2]:
import MeCab

tagger = MeCab.Tagger("-Owakati") 
text = "この木なんの木気になる木"
print(tagger.parse(text).split())

['この', '木', 'な', 'ん', 'の', '木', '気', 'に', 'なる', '木']


**形態素解析をするメッセージデータのアップロード**

今回は、前回取得したメッセージデータを採用。
https://drive.google.com/file/d/1r4DWXDwqluxhxnnaFS9Pdoh5lNND0Ua-/view?usp=sharing


1.ダウンロード

[![Image from Gyazo](https://i.gyazo.com/6b7ef42bc8f28f9678855d92fb20c09d.png)](https://gyazo.com/6b7ef42bc8f28f9678855d92fb20c09d)

2.アップロード

[![Image from Gyazo](https://i.gyazo.com/f030ea871cf214c9f825ce01837747e2.png)](https://gyazo.com/f030ea871cf214c9f825ce01837747e2)

In [9]:
import pandas as pd
import os 
df = pd.read_csv(os.getcwd() +"/(株)オービック.csv", index_col=0)
df

,Date,Text
0,2021-02-03 23:03:08,同じCMをいつまでも流す嫌味な会社
1,2021-02-03 20:02:05,ありがとう、確かに空売りしてる人少ないね
2,2021-02-03 19:03:00,信用の貸借倍率だよ
3,2021-02-03 18:00:03,これって何の倍率なん？
4,2021-02-03 17:04:00,倍率をみれば 買うのは とにかく 今じゃない
...,...,...
182,2021-01-06 09:02:06,降りる時、教えてください。
183,2021-01-06 09:01:07,そー言ってもらえるとうれしい\nありがとう
184,2021-01-06 09:01:00,キリンさん来たら上がったよ！
185,2021-01-06 09:00:05,あれっ、何かいいことあったん？


In [ ]:
# text = "この木なんの木気になる木"
# print(tagger.parse(text).split())

In [10]:
func = lambda x: tagger.parse(x).split()
df["Text_splited"] = df["Text"].map(func)
df

,Date,Text,Text_splited
0,2021-02-03 23:03:08,同じCMをいつまでも流す嫌味な会社,"[同じ, CM, を, いつ, まで, も, 流す, 嫌味, な, 会社]"
1,2021-02-03 20:02:05,ありがとう、確かに空売りしてる人少ないね,"[ありがとう, 、, 確か, に, 空売り, し, てる, 人, 少ない, ね]"
2,2021-02-03 19:03:00,信用の貸借倍率だよ,"[信用, の, 貸借, 倍率, だ, よ]"
3,2021-02-03 18:00:03,これって何の倍率なん？,"[これ, って, 何, の, 倍率, なん, ？]"
4,2021-02-03 17:04:00,倍率をみれば 買うのは とにかく 今じゃない,"[倍率, を, みれ, ば, 買う, の, は, とにかく, 今, じゃ, ない]"
...,...,...,...
182,2021-01-06 09:02:06,降りる時、教えてください。,"[降りる, 時, 、, 教え, て, ください, 。]"
183,2021-01-06 09:01:07,そー言ってもらえるとうれしい\nありがとう,"[そ, ー, 言っ, て, もらえる, と, うれしい, ありがとう]"
184,2021-01-06 09:01:00,キリンさん来たら上がったよ！,"[キリン, さん, 来, たら, 上がっ, た, よ, ！]"
185,2021-01-06 09:00:05,あれっ、何かいいことあったん？,"[あれ, っ, 、, 何, か, いい, こと, あっ, た, ん, ？]"


# 2. メッセージをplus(1>=N), minus(N<= -1 ), normal(±0) に分類

今回は東北大学の乾・鈴木研究室のページで公開されている日本語評価極性辞書を使った

Sentiment Analysis (いわゆるネガポジ判定) ライブラリ oseti を採用。
https://qiita.com/yukinoi/items/46aa016d83bb0e64f598

In [5]:
!pip install oseti

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 133kB 8.2MB/s 
  Created wheel for oseti: filename=oseti-0.2-cp37-none-any.whl size=74357 sha256=fa3473d49b96d6050202c559bd93eb1dd572ab6bccb1fdf30254cb2c920ce420
  Stored in directory: /root/.cache/pip/wheels/3f/a0/b8/306332f20803a7af8bbb35ab3ddaf625dfef62076a6372eec7
  Created wheel for sengiri: filename=sengiri-0.2.1-cp37-none-any.whl size=4352 sha256=3e7b6e99ccea50036d3cd93c4f937627b6fec4cd2a20abd341e699334fa9dce8
  Stored in directory: /root/.cache/pip/wheels/dd/c1/36/d123492db6ec7445fe4124351505e1150a35e03e5c08f3dad0
  Created wheel for neologdn: filename=neologdn-0.4-cp37-cp37m-linux_x86_64.whl size=188247 sha256=46d29a73e88879cbbfce32ac46e21c6209eb8b8059be90adde15e87d97ca0a94
  Stored in directory: /root/.cache/pip/wheels/b3/6f/d4/c132b4e7aef22019e307e7673d97010644c9c15f28c0d0b018
Successfully built oseti sengiri neologdn


In [11]:
import oseti

analyzer = oseti.Analyzer()

analyzer.analyze('天国で待ってる。')

analyzer.analyze('遅刻したけど楽しかったし嬉しかった。すごく充実した！')

[0.3333333333333333, 1.0]

In [12]:
func = lambda x: analyzer.analyze(x)[0]
df["Text_emotion_val"] = df["Text"].map(func)
df

,Date,Text,Text_splited,Text_emotion_val
0,2021-02-03 23:03:08,同じCMをいつまでも流す嫌味な会社,"[同じ, CM, を, いつ, まで, も, 流す, 嫌味, な, 会社]",-1.0
1,2021-02-03 20:02:05,ありがとう、確かに空売りしてる人少ないね,"[ありがとう, 、, 確か, に, 空売り, し, てる, 人, 少ない, ね]",-1.0
2,2021-02-03 19:03:00,信用の貸借倍率だよ,"[信用, の, 貸借, 倍率, だ, よ]",1.0
3,2021-02-03 18:00:03,これって何の倍率なん？,"[これ, って, 何, の, 倍率, なん, ？]",0.0
4,2021-02-03 17:04:00,倍率をみれば 買うのは とにかく 今じゃない,"[倍率, を, みれ, ば, 買う, の, は, とにかく, 今, じゃ, ない]",0.0
...,...,...,...,...
182,2021-01-06 09:02:06,降りる時、教えてください。,"[降りる, 時, 、, 教え, て, ください, 。]",0.0
183,2021-01-06 09:01:07,そー言ってもらえるとうれしい\nありがとう,"[そ, ー, 言っ, て, もらえる, と, うれしい, ありがとう]",0.0
184,2021-01-06 09:01:00,キリンさん来たら上がったよ！,"[キリン, さん, 来, たら, 上がっ, た, よ, ！]",0.0
185,2021-01-06 09:00:05,あれっ、何かいいことあったん？,"[あれ, っ, 、, 何, か, いい, こと, あっ, た, ん, ？]",1.0


# 3. plus, minus, normalに分けたメッセージにある単語を多い順にソートする

In [13]:
plus = df[df['Text_emotion_val'] > 0]
plus = plus.copy()
normal = df[df['Text_emotion_val'] == 0]
normal = normal.copy()
minus = df[df['Text_emotion_val'] < 0]
minus = minus.copy()

In [14]:
plus 

,Date,Text,Text_splited,Text_emotion_val
2,2021-02-03 19:03:00,信用の貸借倍率だよ,"[信用, の, 貸借, 倍率, だ, よ]",1.000000
8,2021-02-02 11:03:02,少し、上向きに動くのか？\n今日こそ動いてほしい。,"[少し, 、, 上向き, に, 動く, の, か, ？, 今日, こそ, 動い, て, ほし...",1.000000
10,2021-02-02 08:04:06,今日はオービックリの2万円台復帰でお願いします,"[今日, は, オービックリ, の, 2, 万, 円, 台, 復帰, で, お願い, し, ます]",1.000000
13,2021-02-01 22:00:06,目標株価 17650円\n2000円の割高,"[目標, 株価, 17650, 円, 2000, 円, の, 割高]",1.000000
18,2021-02-01 14:02:02,あと１０００円位落ちないと\n\n本格的に反転はできないんじゃないの？,"[あと, １, ０, ０, ０, 円, 位, 落ち, ない, と, 本格, 的, に, 反転...",1.000000
24,2021-02-01 11:03:02,憧れていた値嵩株。21000円で、買ったら含み損拡大中。株辛い。,"[憧れ, て, い, た, 値嵩株, 。, 21000, 円, で, 、, 買っ, たら, ...",1.000000
25,2021-02-01 10:03:05,下で集めて行ったら、遠くない将来素晴らしいことになりそう。,"[下, で, 集め, て, 行っ, たら, 、, 遠く, ない, 将来, 素晴らしい, こと...",1.000000
34,2021-01-29 11:01:09,上がって売れば、良かったのか…\nまた真っ逆さま⁈\n\nどこかに遊ばれていますね…泣,"[上がっ, て, 売れ, ば, 、, 良かっ, た, の, か, …, また, 真っ逆さま,...",1.000000
41,2021-01-29 09:00:00,もしかしたら今日は暇になるん？,"[もしか, し, たら, 今日, は, 暇, に, なる, ん, ？]",1.000000
42,2021-01-29 08:00:01,今日は伸びますかね…\n期待しています。,"[今日, は, 伸び, ます, か, ね, …, 期待, し, て, い, ます, 。]",1.000000


In [15]:
normal

,Date,Text,Text_splited,Text_emotion_val
3,2021-02-03 18:00:03,これって何の倍率なん？,"[これ, って, 何, の, 倍率, なん, ？]",0.0
4,2021-02-03 17:04:00,倍率をみれば 買うのは とにかく 今じゃない,"[倍率, を, みれ, ば, 買う, の, は, とにかく, 今, じゃ, ない]",0.0
5,2021-02-02 20:00:02,61倍,"[61, 倍]",0.0
6,2021-02-02 16:05:08,ここは、毎日株価をチェックするような銘柄じゃないと思う。数ヶ月とか数年単価で大きくなっていく...,"[ここ, は, 、, 毎日, 株価, を, チェック, する, よう, な, 銘柄, じゃ,...",0.0
7,2021-02-02 16:01:04,動きません…泣,"[動き, ませ, ん, …, 泣]",0.0
...,...,...,...,...
181,2021-01-06 09:03:01,まだはもうなり…\nもうはまだなり…\nまだなり？,"[まだ, は, もう, なり, …, もう, は, まだ, なり, …, まだ, なり, ？]",0.0
182,2021-01-06 09:02:06,降りる時、教えてください。,"[降りる, 時, 、, 教え, て, ください, 。]",0.0
183,2021-01-06 09:01:07,そー言ってもらえるとうれしい\nありがとう,"[そ, ー, 言っ, て, もらえる, と, うれしい, ありがとう]",0.0
184,2021-01-06 09:01:00,キリンさん来たら上がったよ！,"[キリン, さん, 来, たら, 上がっ, た, よ, ！]",0.0


In [16]:
minus

,Date,Text,Text_splited,Text_emotion_val
0,2021-02-03 23:03:08,同じCMをいつまでも流す嫌味な会社,"[同じ, CM, を, いつ, まで, も, 流す, 嫌味, な, 会社]",-1.000000
1,2021-02-03 20:02:05,ありがとう、確かに空売りしてる人少ないね,"[ありがとう, 、, 確か, に, 空売り, し, てる, 人, 少ない, ね]",-1.000000
11,2021-02-02 08:03:08,オービックって無借金経営だろ\n株価 株主対策要らねえわ,"[オービック, って, 無, 借金, 経営, だろ, 株価, 株主, 対策, 要ら, ねえ, わ]",-1.000000
14,2021-02-01 21:04:08,終わってる 魅力ない企業,"[終わっ, てる, 魅力, ない, 企業]",-1.000000
15,2021-02-01 20:01:01,"割高やね（笑）俺の邪道指数では15,000円です?????沖縄もクラスターでたーいへんです❢","[割高, や, ね, （, 笑, ）, 俺, の, 邪道, 指数, で, は, 15, ,,...",-1.000000
19,2021-02-01 14:01:06,悲観的だと寿命が短いらしいよ\nコップに半分しか水がない ×\nコップに半分も水が残ってる ◯,"[悲観, 的, だ, と, 寿命, が, 短い, らしい, よ, コップ, に, 半分, し...",-1.000000
20,2021-02-01 14:00:05,しそーだけど、絶対しないような気がする。,"[しそ, ー, だ, けど, 、, 絶対, し, ない, よう, な, 気, が, する, 。]",-1.000000
26,2021-02-01 10:02:06,ボリンジャー3σを大幅に割る下落。短期では反発すると思いますが、先日の好決算でも下げと調整に...,"[ボリンジャー, 3, σ, を, 大幅, に, 割る, 下落, 。, 短期, で, は, ...",-1.000000
44,2021-01-28 18:05:09,下落トレンドへ\nマーケットも飽和状態,"[下落, トレンド, へ, マーケット, も, 飽和, 状態]",-1.000000
46,2021-01-28 16:03:02,まだ落ちる,"[まだ, 落ちる]",-1.000000


In [17]:
import MeCab
tagger = MeCab.Tagger("-Ochasen") 

def convert_text_nouns(text) :
    nouns = [line.split()[0] for line in tagger.parse(text).splitlines() if "名詞" in line.split()[-1]]
    return nouns 

for data in [plus, normal, minus]:
    data["splited_data"] = data["Text"].map(convert_text_nouns)
    
    word_counts = {} 
    for values in data["splited_data"] :
        for val in values :
            if  val  in word_counts :
                word_counts[val] += 1 
            else:
                word_counts[val] = 1 
            
    
    score_sorted = sorted(word_counts.items(), key=lambda x:x[1], reverse=True)
    
    print(score_sorted)
    print("-" * 100)

[('円', 14), ('決算', 13), ('の', 12), ('今日', 9), ('株価', 9), ('業績', 9), ('株', 8), ('企業', 6), ('連続', 6), ('ん', 5), ('銘柄', 5), ('投資', 5), ('年', 5), ('的', 4), ('オービック', 4), ('よう', 4), ('笑', 4), ('17', 4), ('ここ', 4), ('最高', 4), ('益', 4), ('間違い', 4), ('０', 3), ('こと', 3), ('そう', 3), ('最近', 3), ('日経', 3), ('指数', 3), ('ニュース', 3), ('問題', 3), ('キリン', 3), ('明日', 3), ('1', 3), ('配当', 3), ('分割', 3), ('目標', 2), ('値嵩株', 2), ('含み損', 2), ('どこ', 2), ('泣', 2), ('期待', 2), ('ケース', 2), ('機関', 2), ('これ', 2), ('方', 2), ('リーク', 2), ('何', 2), ('気分', 2), ('時', 2), ('競馬', 2), (',', 2), ('俺', 2), ('チャート', 2), ('時間', 2), ('確実', 2), ('上昇', 2), ('確信', 2), ('増配', 2), ('超', 2), ('買い', 2), ('25', 2), ('当たり', 2), ('金', 2), ('♡', 2), ('会社', 2), ('日本電産', 2), ('好', 2), ('さん', 2), ('素直', 2), ('意欲', 2), ('性', 2), ('家', 2), ('右肩', 2), ('上がり', 2), ('さ', 2), ('‼', 2), ('残', 2), ('信用', 1), ('貸借', 1), ('倍率', 1), ('上向き', 1), ('オービックリ', 1), ('2', 1), ('万', 1), ('台', 1), ('復帰', 1), ('お願い', 1), ('17650', 1), ('2000', 1), ('割高', 1), ('あと',

### Reference
- https://note.com/junmaeda/n/n39b8b286da13

- https://qiita.com/g-k/items/1b7c765fa6520297ca7c

# Closing remarks


[![Image from Gyazo](https://i.gyazo.com/301c46663a66c700eea1aa066e4a0e2b.png)](https://gyazo.com/301c46663a66c700eea1aa066e4a0e2b)

[![Image from Gyazo](https://i.gyazo.com/27cc5af78af3ef4d7a4244f493227c8d.png)](https://gyazo.com/27cc5af78af3ef4d7a4244f493227c8d)

[![Image from Gyazo](https://i.gyazo.com/64f2c5739bbd4f6ac04c0b6e93ce3ee2.png)](https://gyazo.com/64f2c5739bbd4f6ac04c0b6e93ce3ee2)

[![Image from Gyazo](https://i.gyazo.com/ee800b74e2f49d75013c18e0705ea149.png)](https://gyazo.com/ee800b74e2f49d75013c18e0705ea149)


<h4 style="text-align:center;"> みなさんも実際にプログラムを作成してみてください!!!! 良いデータサイエンスライフを!!!!💪 </h4>


[![Image from Gyazo](https://i.gyazo.com/d9b57ca7fe3bc438fa9a7c60ca027940.png)](https://gyazo.com/d9b57ca7fe3bc438fa9a7c60ca027940)

https://github.com/Densuke-fitness

---
---
---
---

## Additional implementations: Tamura Challenge

せっかくなので、手動でデータを分類することを見越して下記のTODOが実行可能な状態に持っていけるようにします。

今回はディスカッションしながらゴールに辿りこうと思います。


1. 第１段階、第２段階の各段階で収集したメッセージデータの中から1000個のメッセージをランダムに選択し、1つのdataframeに保存
    1.  ランダムに企業コードを選択するプログラムを実行し、選択された企業名からcsvデータを読み込む
    2. 読み込まれたcsvデータのメッセージの項目から適当に1つ選択する
    3. 1, 2を1000回繰り返す

In [19]:
!git clone https://github.com/Densuke-fitness/KawaseSeminarProgrammingMaterials.git TODO

Cloning into 'TODO'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 116 (delta 47), reused 89 (delta 23), pack-reused 0
Receiving objects: 100% (116/116), 5.98 MiB | 25.86 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [25]:
DIR = "/content/TODO/Kawase Seminar06/todo-data"

import pandas as pd
pd.read_csv(DIR+"/data1.csv")

,Unnamed: 0,Date,Text
0,0,2021-02-03 23:03:08,同じCMをいつまでも流す嫌味な会社
1,1,2021-02-03 20:02:05,ありがとう、確かに空売りしてる人少ないね
2,2,2021-02-03 19:03:00,信用の貸借倍率だよ
3,3,2021-02-03 18:00:03,これって何の倍率なん？
4,4,2021-02-03 17:04:00,倍率をみれば 買うのは とにかく 今じゃない
...,...,...,...
182,182,2021-01-06 09:02:06,降りる時、教えてください。
183,183,2021-01-06 09:01:07,そー言ってもらえるとうれしい\nありがとう
184,184,2021-01-06 09:01:00,キリンさん来たら上がったよ！
185,185,2021-01-06 09:00:05,あれっ、何かいいことあったん？



1. 第１段階、第２段階の各段階で収集したメッセージデータの中から1000個のメッセージをランダムに選択し、1つのdataframeに保存
    1.  ランダムに企業コードを選択するプログラムを実行し、選択された企業名からcsvデータを読み込む
    2. 読み込まれたcsvデータのメッセージの項目から適当に1つ選択する
    3. 1, 2を1000回繰り返しデータを保存する

## Challenge.コードを実装するために話し合い、1人の画面を共有して実際に書いてみよう

**プログラミングの話し合いのコツ**

- 目的を明確にする (ex: 今回はtodoの要件を満たすことが目的)
- 議論しているポイントをその都度意識する(ex: todoごとの1〜３それぞれの処理についての動きを見失わない)
- 大きく解こうとせずに細かく分ける(ex, ランダムに企業を選択する: ランダム, 企業を選択)
- その都度調べる(探したいことは基本インターネットにに転がってます、ドキュメントなら尚更です)

In [43]:
# DIR = "/content/TODO/Kawase Seminar06/todo-data"
# コードを実装せよ(必要であれば関数を定義し可読性を上げるとポイント高いです)
#-------------------------
##### 1: ランダムに企業コードを選択するプログラムを実行し、選択された企業名からcsvデータを読み込む ######

In [42]:
##### 2: 2読み込まれたcsvデータのメッセージの項目から適当に1つ選択する ######

In [41]:
##### 3: 1, 2を1000回繰り返す #####